# Import the Packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_sample = 0.1*mnist_info.splits['train'].num_examples
num_validation_sample = tf.cast(num_validation_sample, tf.int64)

num_test_sample = mnist_info.splits['test'].num_examples
num_test_sample = tf.cast(num_test_sample, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

# Scaling
scaled_train_and_validation_data = mnist_train.map(scale) 
test_data = mnist_test.map(scale)

In [4]:
buffer_size = 10000

#Shuffling
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_sample)
train_data = shuffled_train_and_validation_data.skip(num_validation_sample)

In [5]:
# Batching
batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_sample)
test_data = test_data.batch(num_test_sample)

In [6]:
validation_input, validation_targets = next(iter(validation_data))

# Model

## Outline the Model

In [7]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)), # Flatten tensor into a vector
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), # Applying first hidden layer
    tf.keras.layers.Dense(128, activation = 'tanh'), # Applying second hidden layer
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax') # Applying softmax as activation because we want probablity
])

## Choosing the optimizer and the loss function

In [8]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Training

In [9]:
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_input, validation_targets), validation_steps = 1, verbose = 2)

Epoch 1/5
540/540 - 7s - loss: 0.2772 - accuracy: 0.9206 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 4s - loss: 0.1021 - accuracy: 0.9685 - val_loss: 0.0916 - val_accuracy: 0.9740
Epoch 3/5
540/540 - 4s - loss: 0.0678 - accuracy: 0.9791 - val_loss: 0.0667 - val_accuracy: 0.9808
Epoch 4/5
540/540 - 4s - loss: 0.0502 - accuracy: 0.9848 - val_loss: 0.0559 - val_accuracy: 0.9835
Epoch 5/5
540/540 - 4s - loss: 0.0396 - accuracy: 0.9878 - val_loss: 0.0430 - val_accuracy: 0.9878


In [10]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 984ms/step - loss: 0.0683 - accuracy: 0.9808
Test loss: 0.07. Test accuracy: 98.08%
